## 3.2.1 Logistic Regression Example with TF-IDF

### TF-IDF Feature Example

In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
# 전처리 한 데이터를 불러온다.
train_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_DATA )

In [4]:
# 리뷰값과 라벨값을 각각 따로 리스트로 지정해둔다.
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [5]:
# 객체 생성
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000) 

# min_df: 설정한 값보다 특정 토큰의 df 값이 더 적게 나오면 벡터화 과정에서 제가한다는 의미
# analyzer: 분석하기 위한 기준 단위, char의 경우 문자 하나를 단위로 한다
# sublinear_tf: 문서의 단어 빈도수에 대한 스무딩 여부를 설정하는 값
# ngram_range: 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인가를 보는 인자
# max_features: 각 벡터의 최대 길이, 특징의 최대 길이

# 전체 문장에 대한 특징 벡터 데이터 X를 생성
X = vectorizer.fit_transform(reviews)
# 라벨값 넘파이 배열로 생성
y = np.array(sentiments)

In [6]:
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 17862755 stored elements in Compressed Sparse Row format>

In [7]:
features = vectorizer.get_feature_names()

In [8]:
# 학습 데이터의 20%를 검증 데이터로 따로 분리 
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [9]:
# LogisticRegression 클래스 객체 선언
lgs = LogisticRegression(class_weight='balanced') 

# class_weighted = 'balanced' : 각 라벨에 대해 균형 있게 학습할 수 있게 지정

# 모델 학습 진행
lgs.fit(X_train, y_train) 

LogisticRegression(class_weight='balanced')

In [10]:
predicted = lgs.predict(X_eval)

In [11]:
# 학습한 모델 성능 측정
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

Accuracy: 0.860000


In [12]:
# 캐글에 제출할 파일 생성

# 파일명 설정
TEST_CLEAN_DATA = 'test_clean.csv'
# 테스트 데이터를 판다스로 불러온다.
test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [13]:
# TF-IDF값으로 변환
testDataVecs = vectorizer.transform(test_data['review'])

In [14]:
# 예측값 출력
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [15]:
# 캐글에 제출하기 위한 파일 형태로 저장
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)